In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dtime
import math
import seaborn as sns
from math import radians
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from scipy.cluster.vq import kmeans2, whiten
from sklearn.cluster import DBSCAN, KMeans
from sklearn.metrics import roc_auc_score
from sklearn.grid_search import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk.data
import nltk
from sklearn import metrics
from sklearn import cross_validation
from sklearn.model_selection import cross_validate
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, roc_auc_score, precision_score
from sklearn.model_selection import cross_val_predict

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/anaconda3/li

In [2]:
# Converting txt data into Dataframe

In [3]:
text_file = open("train.txt", "r",encoding="utf8", errors='ignore')
lines = text_file.readlines()
text_file.close()

In [4]:
type(lines)

list

In [5]:
import pandas as pd
import numpy as np
print(len(lines))

882


In [6]:
text_file = open("label.txt", "r",encoding="utf8", errors='ignore')
label = text_file.readlines()
text_file.close()
# print(len(label))
# print(lines[0])

In [7]:
df_train = pd.DataFrame(lines)
df_labels = pd.DataFrame(label)

In [8]:
print(df_train.shape)
print(df_labels.shape)

(882, 1)
(882, 1)


In [9]:
print(df_train[0:2])

                                                   0
0  20030006121    Passive radio frequency identif...
1  20030009666    Methods and apparatus for effic...


In [10]:
#split label and text into different columns
df_train['ID'] = df_train.iloc[:,0].str[:11]

In [11]:
df_train['text'] = df_train.iloc[:,0].str[12:]

In [12]:
print(df_train.shape)
print(df_train[0:2])

(882, 3)
                                                   0           ID  \
0  20030006121    Passive radio frequency identif...  20030006121   
1  20030009666    Methods and apparatus for effic...  20030009666   

                                                text  
0     Passive radio frequency identification syst...  
1     Methods and apparatus for efficient computa...  


In [13]:
#drop raw data column
df_train = df_train.drop(df_train.columns[0], axis=1)

In [14]:
print(df_train.shape)
print(df_train[0:2])

(882, 2)
            ID                                               text
0  20030006121     Passive radio frequency identification syst...
1  20030009666     Methods and apparatus for efficient computa...


In [15]:
#trim leading and trailing whitespaces
df_train['text'] = df_train['text'].str.strip()

In [16]:
print(df_train.loc[0:2])

            ID                                               text
0  20030006121  Passive radio frequency identification system ...
1  20030009666  Methods and apparatus for efficient computatio...
2  20030014337  Systems, methods and computer program products...


In [17]:
print(df_labels.loc[0:2])

                  0
0  20030006121\t8\n
1  20030009666\t0\n
2  20030014337\t8\n


In [18]:
#assiging column name
df_labels.columns = ['raw']

In [19]:
#extracting labels from raw data and creating a new column
df_labels['Label'] = df_labels.iloc[:,0].str[12:]
print(df_labels.loc[0:2])

                raw Label
0  20030006121\t8\n   8\n
1  20030009666\t0\n   0\n
2  20030014337\t8\n   8\n


In [20]:
#extracting id from the raw data column and removing redundant characters from label column
df_labels['ID'] = df_labels['raw'].astype(str).str[:-2].astype(np.int64)
df_labels['Label'] = df_labels['Label'].astype(str).str[:-1].astype(np.int64)
print(df_labels.loc[0:2])

                raw  Label           ID
0  20030006121\t8\n      8  20030006121
1  20030009666\t0\n      0  20030009666
2  20030014337\t8\n      8  20030014337


In [21]:
#drop raw data column
df_labels = df_labels.drop(df_labels.columns[0], axis=1)
print(df_labels.loc[0:2])

   Label           ID
0      8  20030006121
1      0  20030009666
2      8  20030014337


In [22]:
print(type(df_train.ix[0,"ID"]))
print(type(df_labels.ix[0,"ID"]))

<class 'str'>
<class 'numpy.int64'>


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [23]:
#Ensure both the ID columns have the same data type to avoid any error while merging the data frames
df_labels["ID"] = df_labels["ID"].astype(str)

In [24]:
#merging two data frames
df = pd.merge(df_train,
                 df_labels,
                 on='ID')

print(df.loc[0:2])

            ID                                               text  Label
0  20030006121  Passive radio frequency identification system ...      8
1  20030009666  Methods and apparatus for efficient computatio...      0
2  20030014337  Systems, methods and computer program products...      8


In [ ]:
#Save the dataframe as csv file
df.to_csv('data.csv')

# Data Processing

In [25]:
#Lower Case
df['text'] = df['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['text'].head()

0    passive radio frequency identification system ...
1    methods and apparatus for efficient computatio...
2    systems, methods and computer program products...
3    system and method for electronically readable ...
4    system and method for automated process of dea...
Name: text, dtype: object

In [26]:
#Remove Punctuation
df['text'] = df['text'].str.replace('[^\w\s]','')
df['text'].head()

0    passive radio frequency identification system ...
1    methods and apparatus for efficient computatio...
2    systems methods and computer program products ...
3    system and method for electronically readable ...
4    system and method for automated process of dea...
Name: text, dtype: object

In [27]:
#Remove Stopwords
#import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['text'].head()

0    passive radio frequency identification system ...
1    methods apparatus efficient computation oneway...
2    systems methods computer program products perf...
3    system method electronically readable card pow...
4    system method automated process deal structuri...
Name: text, dtype: object

In [28]:
#Identify top 50 common words
freq = pd.Series(' '.join(df['text']).split()).value_counts()[:50]
freq

may              165856
data              80094
user              78796
system            77428
transaction       75169
one               67899
payment           67298
device            65938
information       65207
account           62264
example           45461
mobile            39757
andor             37496
card              36974
associated        33185
embodiments       30355
embodiment        30331
network           28984
method            28769
include           28730
financial         28681
eg                28583
fig               28547
computer          27933
used              27477
invention         26462
wherein           25988
server            25987
number            25977
merchant          25829
customer          25759
first             25694
application       24325
based             23460
also              23265
step              22182
credit            22125
using             21512
request           21404
communication     20870
service           20782
said            

In [29]:
#top 50 common word removal
freq = list(freq.index)
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
df['text'].head()

0    passive radio frequency identification identif...
1    methods apparatus efficient computation oneway...
2    systems methods program products performing ge...
3    electronically readable power source improved ...
4    automated process deal structuring automated d...
Name: text, dtype: object

In [30]:
#Tokenization
from textblob import TextBlob
TextBlob(df['text'][1]).words

WordList(['methods', 'apparatus', 'efficient', 'computation', 'oneway', 'chains', 'cryptographic', 'applications', 'techniques', 'disclosed', 'efficient', 'computation', 'consecutive', 'values', 'oneway', 'chains', 'oneway', 'graphs', 'cryptographic', 'applications', 'oneway', 'chain', 'graph', 'chain', 'length', 'positions', 'i1', '2', 'corresponding', 'vl', 'therewith', 'vi', 'given', 'vih', 'vl1', 'given', 'hash', 'function', 'oneway', 'function', 'h', 'initial', 'distribution', 'helper', 'values', 'stored', 'oneway', 'chain', 'length', 'positions', 'given', 'i2j', '0jlog2', 'given', 'output', 'values', 'vi', 'current', 'position', 'oneway', 'chain', 'computed', 'utilizing', 'helper', 'previously', 'stored', 'another', 'position', 'oneway', 'chain', 'current', 'position', 'endpoint', 'chain', 'computation', 'given', 'output', 'positions', 'helper', 'values', 'adjusted', 'facilitate', 'computation', 'subsequent', 'output', 'values', 'advantageously', 'storagecomputation', 'product', 

In [31]:
#Stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()
df['train'] = df['text'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

In [32]:
#Term Frequency
tf1 = (df['text'][1:2]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf1.columns = ['words','tf']
tf1

,words,tf
0,chain,203
1,oneway,105
2,values,103
3,peg,88
4,position,81
5,helper,75
6,pegs,65
7,computation,61
8,output,61
9,function,59


In [33]:
#Inverse document frequency
for i,word in enumerate(tf1['words']):
    tf1.loc[i, 'idf'] = np.log(df.shape[0]/(len(df[df['text'].str.contains(word)])))
tf1

,words,tf,idf
0,chain,203,1.388565
1,oneway,105,2.975530
2,values,103,0.790728
3,peg,88,3.255832
4,position,81,0.561602
5,helper,75,3.837753
6,pegs,65,6.782192
7,computation,61,1.758312
8,output,61,0.459627
9,function,59,0.097580


In [34]:
#Tf-idf method
tf1['tfidf'] = tf1['tf'] * tf1['idf']
tf1.shape

(1223, 4)

In [35]:
#Tokenization
df['train_text'] = df['train'].apply(nltk.word_tokenize)

In [36]:
df.head()

,ID,text,Label,train,train_text
0,20030006121,passive radio frequency identification identif...,8,passiv radio frequenc identif identifi track c...,"[passiv, radio, frequenc, identif, identifi, t..."
1,20030009666,methods apparatus efficient computation oneway...,0,method apparatu effici comput oneway chain cry...,"[method, apparatu, effici, comput, oneway, cha..."
2,20030014337,systems methods program products performing ge...,8,system method program product perform gener co...,"[system, method, program, product, perform, ge..."
3,20030019942,electronically readable power source improved ...,8,electron readabl power sourc improv contain el...,"[electron, readabl, power, sourc, improv, cont..."
4,20030033242,automated process deal structuring automated d...,6,autom process deal structur autom deal custom ...,"[autom, process, deal, structur, autom, deal, ..."


In [37]:
#counting different labels
df.groupby('Label').count()

,ID,text,train,train_text
Label,,,,
0,274,274,274,274
1,84,84,84,84
2,77,77,77,77
3,61,61,61,61
4,37,37,37,37
5,42,42,42,42
6,45,45,45,45
7,34,34,34,34
8,228,228,228,228


In [38]:
#setting X & y for spliting the dataset
X = df['train']
y = df['Label']

In [39]:
#Spliting data into train & test set, P.S: we dont really use it
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = False)


In [40]:
#Computing dimension of the document term matrix, training set
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',
stop_words= 'english',ngram_range=(1,1))
train_vect = tfidf.fit_transform(X_train)
train_vect

<705x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 329789 stored elements in Compressed Sparse Row format>

In [41]:
#Computing dimension of the document term matrix, testing set
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',
stop_words= 'english',ngram_range=(1,1))
test_vect = tfidf.fit_transform(X_test)
test_vect

<177x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 88522 stored elements in Compressed Sparse Row format>

In [42]:
#Computing dimension of the document term matrix, whole dataset
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',
stop_words= 'english',ngram_range=(1,1))
all_vect = tfidf.fit_transform(X)
all_vect

<882x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 418800 stored elements in Compressed Sparse Row format>

# Random Forest

In [76]:
RF = RandomForestClassifier(n_estimators=500)

In [77]:
RF.fit(all_vect, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [105]:
# print ('Accuracy Score: ', accuracy_score(y, rf_pred))

In [87]:
scoring = {'acc': 'accuracy',
           'precision': 'precision_macro',
           'recall': 'recall_macro',
           'f1_score' : 'f1_macro'}
rf_scores = cross_validate(RF, all_vect, y, scoring=scoring,
                         cv=10, return_train_score=True)

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py

In [89]:
print(rf_scores.keys())
print(rf_scores['test_acc'].mean())
print(rf_scores['test_precision'].mean())
print(rf_scores['test_recall'].mean())
print(rf_scores['test_f1_score'].mean())

dict_keys(['fit_time', 'score_time', 'test_acc', 'train_acc', 'test_precision', 'train_precision', 'test_recall', 'train_recall', 'test_f1_score', 'train_f1_score'])
0.6371231643650203
0.7049846980488556
0.525672994030482
0.5435493079010157


In [104]:
#Use cross validation predict to make prediction on the entire dataset
rf_pred = cross_val_predict(RF, all_vect, y, cv=10)

#Use classification report to get the precision & recall table, compare with prediction and actual labels
print(classification_report(y,rf_pred))

             precision    recall  f1-score   support

          0       0.62      0.88      0.73       274
          1       0.76      0.37      0.50        84
          2       0.81      0.44      0.57        77
          3       0.81      0.57      0.67        61
          4       0.83      0.41      0.55        37
          5       0.80      0.29      0.42        42
          6       0.71      0.38      0.49        45
          7       0.95      0.62      0.75        34
          8       0.54      0.70      0.61       228

avg / total       0.68      0.64      0.63       882



In [122]:
#Assign the prediction back to dataframe
df['rf_prediction'] = rf_pred

In [123]:
#Count the prediction labels
df.groupby('rf_prediction').count()

,ID,text,Label,train,train_text
rf_prediction,,,,,
0,385,385,385,385,385
1,41,41,41,41,41
2,42,42,42,42,42
3,43,43,43,43,43
4,18,18,18,18,18
5,15,15,15,15,15
6,24,24,24,24,24
7,22,22,22,22,22
8,292,292,292,292,292


In [124]:
#Count the actual labels
df.groupby('Label').count()

,ID,text,train,train_text,rf_prediction
Label,,,,,
0,274,274,274,274,274
1,84,84,84,84,84
2,77,77,77,77,77
3,61,61,61,61,61
4,37,37,37,37,37
5,42,42,42,42,42
6,45,45,45,45,45
7,34,34,34,34,34
8,228,228,228,228,228


# Gradient Boost

In [106]:
GB = GradientBoostingClassifier(learning_rate = 0.25)

In [107]:
GB.fit(all_vect, y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.25, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [57]:
scoring = {'acc': 'accuracy',
           'precision': 'precision_macro',
           'recall': 'recall_macro',
           'f1_score' : 'f1_macro'}
gb_scores = cross_validate(GB, all_vect, y, scoring=scoring,
                         cv=10, return_train_score=True)

/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [58]:
print(gb_scores.keys())
print(gb_scores['test_acc'].mean())
print(gb_scores['test_precision'].mean())
print(gb_scores['test_recall'].mean())
print(gb_scores['test_f1_score'].mean())

dict_keys(['fit_time', 'score_time', 'test_acc', 'train_acc', 'test_precision', 'train_precision', 'test_recall', 'train_recall', 'test_f1_score', 'train_f1_score'])
0.6371013859500783
0.7278177288264145
0.6345988699974208
0.6289157585683471


In [108]:
#Use cross validation predict to make prediction on the entire dataset
gb_pred = cross_val_predict(GB, all_vect, y, cv=10)

#Use classification report to get the precision & recall table, compare with prediction and actual labels
print(classification_report(y,gb_pred))

             precision    recall  f1-score   support

          0       0.65      0.74      0.69       274
          1       0.55      0.39      0.46        84
          2       0.64      0.44      0.52        77
          3       0.76      0.62      0.68        61
          4       0.59      0.70      0.64        37
          5       0.68      0.67      0.67        42
          6       0.73      0.53      0.62        45
          7       0.64      0.79      0.71        34
          8       0.54      0.60      0.57       228

avg / total       0.62      0.62      0.62       882



# Naive Bayes

In [109]:
clf = MultinomialNB().fit(all_vect, y)

In [60]:
scoring = {'acc': 'accuracy',
           'precision': 'precision_macro',
           'recall': 'recall_macro',
           'f1_score' : 'f1_macro'}
nb_scores = cross_validate(clf, all_vect, y, scoring=scoring,
                         cv=10, return_train_score=True)

/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classif

In [61]:
print(nb_scores.keys())
print(nb_scores['test_acc'].mean())
print(nb_scores['test_precision'].mean())
print(nb_scores['test_recall'].mean())
print(nb_scores['test_f1_score'].mean())

dict_keys(['fit_time', 'score_time', 'test_acc', 'train_acc', 'test_precision', 'train_precision', 'test_recall', 'train_recall', 'test_f1_score', 'train_f1_score'])
0.5952837841344678
0.5742715387215398
0.41418035310547385
0.4267011691481966


In [110]:
#Use cross validation predict to make prediction on the entire dataset
nb_pred = cross_val_predict(clf, all_vect, y, cv=10)

#Use classification report to get the precision & recall table, compare with prediction and actual labels
print(classification_report(y,nb_pred))

             precision    recall  f1-score   support

          0       0.58      0.87      0.70       274
          1       0.76      0.26      0.39        84
          2       0.83      0.26      0.40        77
          3       0.85      0.64      0.73        61
          4       0.67      0.05      0.10        37
          5       0.50      0.07      0.12        42
          6       0.58      0.33      0.42        45
          7       0.94      0.47      0.63        34
          8       0.53      0.74      0.62       228

avg / total       0.64      0.59      0.55       882



# KNN

In [111]:
modelknn = KNeighborsClassifier(n_neighbors=5)
modelknn.fit(all_vect,y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [67]:
scoring = {'acc': 'accuracy',
           'precision': 'precision_macro',
           'recall': 'recall_macro',
           'f1_score' : 'f1_macro'}
knn_scores = cross_validate(modelknn, all_vect, y, scoring=scoring,
                         cv=10, return_train_score=True)

/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [68]:
print(knn_scores.keys())
print(knn_scores['test_acc'].mean())
print(knn_scores['test_precision'].mean())
print(knn_scores['test_recall'].mean())
print(knn_scores['test_f1_score'].mean())

dict_keys(['fit_time', 'score_time', 'test_acc', 'train_acc', 'test_precision', 'train_precision', 'test_recall', 'train_recall', 'test_f1_score', 'train_f1_score'])
0.5797076951736007
0.6241607147238207
0.6348628683894384
0.5978273256001948


In [112]:
#Use cross validation predict to make prediction on the entire dataset
knn_pred = cross_val_predict(modelknn, all_vect, y, cv=10)

#Use classification report to get the precision & recall table, compare with prediction and actual labels
print(classification_report(y,knn_pred))

             precision    recall  f1-score   support

          0       0.67      0.69      0.68       274
          1       0.43      0.55      0.48        84
          2       0.43      0.48      0.45        77
          3       0.69      0.74      0.71        61
          4       0.64      0.68      0.66        37
          5       0.47      0.76      0.58        42
          6       0.51      0.67      0.58        45
          7       0.81      0.76      0.79        34
          8       0.57      0.36      0.44       228

avg / total       0.59      0.58      0.57       882



# SVM Linear

In [113]:
svm_clf=SVC(kernel="linear").fit(all_vect,y)

In [71]:
scoring = {'acc': 'accuracy',
           'precision': 'precision_macro',
           'recall': 'recall_macro',
           'f1_score' : 'f1_macro'}
svm_scores = cross_validate(svm_clf, all_vect, y, scoring=scoring,
                         cv=10, return_train_score=True)

/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [72]:
print(svm_scores.keys())
print(svm_scores['test_acc'].mean())
print(svm_scores['test_precision'].mean())
print(svm_scores['test_recall'].mean())
print(svm_scores['test_f1_score'].mean())

dict_keys(['fit_time', 'score_time', 'test_acc', 'train_acc', 'test_precision', 'train_precision', 'test_recall', 'train_recall', 'test_f1_score', 'train_f1_score'])
0.6619155096582132
0.7313168966169523
0.642036804669655
0.6448744542190752


In [114]:
#Use cross validation predict to make prediction on the entire dataset
svm_pred = cross_val_predict(svm_clf, all_vect, y, cv=10)

#Use classification report to get the precision & recall table, compare with prediction and actual labels
print(classification_report(y,svm_pred))

             precision    recall  f1-score   support

          0       0.68      0.82      0.74       274
          1       0.61      0.55      0.58        84
          2       0.69      0.49      0.58        77
          3       0.77      0.75      0.76        61
          4       0.68      0.76      0.72        37
          5       0.60      0.57      0.59        42
          6       0.57      0.58      0.57        45
          7       1.00      0.65      0.79        34
          8       0.60      0.57      0.58       228

avg / total       0.66      0.66      0.66       882



# SVM Non-Linear

In [115]:
svm_non=SVC(kernel="rbf",cache_size=1,class_weight={1:2}).fit(all_vect,y)

In [74]:
scoring = {'acc': 'accuracy',
           'precision': 'precision_macro',
           'recall': 'recall_macro',
           'f1_score' : 'f1_macro'}
non_scores = cross_validate(svm_non, all_vect, y, scoring=scoring,
                         cv=10, return_train_score=True)

/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classif

/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [75]:
print(non_scores.keys())
print(non_scores['test_acc'].mean())
print(non_scores['test_precision'].mean())
print(non_scores['test_recall'].mean())
print(non_scores['test_f1_score'].mean())

dict_keys(['fit_time', 'score_time', 'test_acc', 'train_acc', 'test_precision', 'train_precision', 'test_recall', 'train_recall', 'test_f1_score', 'train_f1_score'])
0.31088197489600444
0.0345424416551116
0.11111111111111112
0.052696694701889334


In [116]:
#Use cross validation predict to make prediction on the entire dataset
non_pred = cross_val_predict(svm_non, all_vect, y, cv=10)

#Use classification report to get the precision & recall table, compare with prediction and actual labels
print(classification_report(y,non_pred))

             precision    recall  f1-score   support

          0       0.31      1.00      0.47       274
          1       0.00      0.00      0.00        84
          2       0.00      0.00      0.00        77
          3       0.00      0.00      0.00        61
          4       0.00      0.00      0.00        37
          5       0.00      0.00      0.00        42
          6       0.00      0.00      0.00        45
          7       0.00      0.00      0.00        34
          8       0.00      0.00      0.00       228

avg / total       0.10      0.31      0.15       882



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Manipulating Labels

In [90]:
#counting labels from original dataframe
df.groupby('Label').count()

,ID,text,train,train_text
Label,,,,
0,274,274,274,274
1,84,84,84,84
2,77,77,77,77
3,61,61,61,61
4,37,37,37,37
5,42,42,42,42
6,45,45,45,45
7,34,34,34,34
8,228,228,228,228


In [91]:
#Create new dataframe that only contains label 0 & 8, both contains 74 labels each
d1 = df[df['Label']==0].sample(74)
d2 = df[df['Label']==8].sample(74)
#Append them into a new dataframe
d3 = d1.append(d2, ignore_index=True)

In [92]:
#Create new dataframe that only contains label 1 to 7
d4 = df[(df['Label'] != 0 ) & (df['Label'] != 8)]
#Append them together
ds = d4.append(d3, ignore_index=True)

In [93]:
#Counting labels from the new dataframe
ds.groupby('Label').count()

,ID,text,train,train_text
Label,,,,
0,74,74,74,74
1,84,84,84,84
2,77,77,77,77
3,61,61,61,61
4,37,37,37,37
5,42,42,42,42
6,45,45,45,45
7,34,34,34,34
8,74,74,74,74


In [94]:
#Randomize the whole dataframe
ds = ds.sample(frac=1)

In [95]:
#Set the target feature column as X1
X1 = ds['train']
y1 = ds['Label']

In [96]:
##Computing new dimension of the document term matrix, whole dataset
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',
stop_words= 'english',ngram_range=(1,1))
new_vect = tfidf.fit_transform(X1)
new_vect

<528x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 258131 stored elements in Compressed Sparse Row format>

# Random Foest (New Data)

In [97]:
RF = RF.fit(new_vect, ds['Label'])

In [83]:
scoring = {'acc': 'accuracy',
           'precision': 'precision_weighted',
           'recall': 'recall_macro',
           'f1_score' : 'f1_macro'}
RF_scores = cross_validate(RF, new_vect, ds['Label'], scoring=scoring,
                         cv=10, return_train_score=True)

In [84]:
print(RF_scores.keys())
print(RF_scores['test_acc'].mean())
print(RF_scores['test_precision'].mean())
print(RF_scores['test_recall'].mean())
print(RF_scores['test_f1_score'].mean())

dict_keys(['fit_time', 'score_time', 'test_acc', 'train_acc', 'test_precision', 'train_precision', 'test_recall', 'train_recall', 'test_f1_score', 'train_f1_score'])
0.7034913120630075
0.7456434025676554
0.7136728395061729
0.71916185270652


In [99]:
#Use cross validation predict to make prediction on the entire dataset
RF_pred = cross_val_predict(RF, new_vect, y1, cv=10)

#Use classification report to get the precision & recall table, compare with prediction and actual labels
print(classification_report(y1,RF_pred))

             precision    recall  f1-score   support

          0       0.55      0.65      0.60        74
          1       0.62      0.71      0.66        84
          2       0.74      0.83      0.78        77
          3       0.94      0.75      0.84        61
          4       0.79      0.92      0.85        37
          5       0.90      0.62      0.73        42
          6       0.79      0.84      0.82        45
          7       1.00      0.62      0.76        34
          8       0.60      0.54      0.57        74

avg / total       0.73      0.71      0.72       528



In [125]:
#Assign the prediction back to dataframe
ds['RF_prediction'] = RF_pred

In [128]:
#Count the prediction labels
ds.groupby('RF_prediction').count()

,ID,text,Label,train,train_text
RF_prediction,,,,,
0,87,87,87,87,87
1,97,97,97,97,97
2,87,87,87,87,87
3,49,49,49,49,49
4,43,43,43,43,43
5,29,29,29,29,29
6,48,48,48,48,48
7,21,21,21,21,21
8,67,67,67,67,67


In [127]:
#Count the actual labels
ds.groupby('Label').count()

,ID,text,train,train_text,RF_prediction
Label,,,,,
0,74,74,74,74,74
1,84,84,84,84,84
2,77,77,77,77,77
3,61,61,61,61,61
4,37,37,37,37,37
5,42,42,42,42,42
6,45,45,45,45,45
7,34,34,34,34,34
8,74,74,74,74,74


# Gradient Boost (New Data)

In [85]:
GB.fit(new_vect, ds['Label'])

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.25, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [86]:
scoring = {'acc': 'accuracy',
           'precision': 'precision_weighted',
           'recall': 'recall_macro',
           'f1_score' : 'f1_macro'}
GB_scores = cross_validate(GB, new_vect, ds['Label'], scoring=scoring,
                         cv=10, return_train_score=True)

In [87]:
print(GB_scores.keys())
print(GB_scores['test_acc'].mean())
print(GB_scores['test_precision'].mean())
print(GB_scores['test_recall'].mean())
print(GB_scores['test_f1_score'].mean())

dict_keys(['fit_time', 'score_time', 'test_acc', 'train_acc', 'test_precision', 'train_precision', 'test_recall', 'train_recall', 'test_f1_score', 'train_f1_score'])
0.6545179428744834
0.678007583152839
0.687067901234568
0.6818498041537258


In [117]:
#Use cross validation predict to make prediction on the entire dataset
GB_pred = cross_val_predict(GB, new_vect, y1, cv=10)

#Use classification report to get the precision & recall table, compare with prediction and actual labels
print(classification_report(y1,GB_pred))

             precision    recall  f1-score   support

          0       0.54      0.61      0.57        74
          1       0.68      0.63      0.65        84
          2       0.69      0.68      0.68        77
          3       0.89      0.80      0.84        61
          4       0.76      0.86      0.81        37
          5       0.70      0.67      0.68        42
          6       0.85      0.76      0.80        45
          7       0.77      0.79      0.78        34
          8       0.48      0.51      0.50        74

avg / total       0.69      0.68      0.68       528



# Naive Bayes (New Data)

In [88]:
clf = MultinomialNB().fit(new_vect, ds['Label'])

In [89]:
scoring = {'acc': 'accuracy',
           'precision': 'precision_weighted',
           'recall': 'recall_macro',
           'f1_score' : 'f1_macro'}
NB_scores = cross_validate(clf, new_vect, ds['Label'], scoring=scoring,
                         cv=10, return_train_score=True)

/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [90]:
print(NB_scores.keys())
print(NB_scores['test_acc'].mean())
print(NB_scores['test_precision'].mean())
print(NB_scores['test_recall'].mean())
print(NB_scores['test_f1_score'].mean())

dict_keys(['fit_time', 'score_time', 'test_acc', 'train_acc', 'test_precision', 'train_precision', 'test_recall', 'train_recall', 'test_f1_score', 'train_f1_score'])
0.630537844064053
0.6502291986570286
0.6463756613756614
0.638160909933615


In [118]:
#Use cross validation predict to make prediction on the entire dataset
NB_pred = cross_val_predict(clf, new_vect, y1, cv=10)

#Use classification report to get the precision & recall table, compare with prediction and actual labels
print(classification_report(y1,NB_pred))

             precision    recall  f1-score   support

          0       0.61      0.36      0.46        74
          1       0.55      0.79      0.65        84
          2       0.48      0.81      0.60        77
          3       0.80      0.77      0.78        61
          4       0.83      0.68      0.75        37
          5       0.81      0.69      0.74        42
          6       0.70      0.82      0.76        45
          7       0.95      0.56      0.70        34
          8       0.50      0.26      0.34        74

avg / total       0.65      0.63      0.61       528



# KNN (New Data)

In [91]:
modelknn = KNeighborsClassifier(n_neighbors=5).fit(new_vect, ds['Label'])

In [92]:
scoring = {'acc': 'accuracy',
           'precision': 'precision_weighted',
           'recall': 'recall_macro',
           'f1_score' : 'f1_macro'}
KNN_scores = cross_validate(modelknn, new_vect, ds['Label'], scoring=scoring,
                         cv=10, return_train_score=True)

/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [93]:
print(KNN_scores.keys())
print(KNN_scores['test_acc'].mean())
print(KNN_scores['test_precision'].mean())
print(KNN_scores['test_recall'].mean())
print(KNN_scores['test_f1_score'].mean())

dict_keys(['fit_time', 'score_time', 'test_acc', 'train_acc', 'test_precision', 'train_precision', 'test_recall', 'train_recall', 'test_f1_score', 'train_f1_score'])
0.602845444334372
0.5826921379998932
0.6566754850088183
0.609897356589823


In [119]:
#Use cross validation predict to make prediction on the entire dataset
KNN_pred = cross_val_predict(modelknn, new_vect, y1, cv=10)

#Use classification report to get the precision & recall table, compare with prediction and actual labels
print(classification_report(y1,KNN_pred))

             precision    recall  f1-score   support

          0       0.53      0.38      0.44        74
          1       0.55      0.67      0.61        84
          2       0.52      0.58      0.55        77
          3       0.78      0.80      0.79        61
          4       0.61      0.73      0.67        37
          5       0.61      0.93      0.74        42
          6       0.67      0.91      0.77        45
          7       0.71      0.79      0.75        34
          8       0.39      0.09      0.15        74

avg / total       0.58      0.60      0.57       528



# SVM Linear (New Data)

In [94]:
svm_clf=SVC(kernel="linear").fit(new_vect,ds['Label'])

In [95]:
scoring = {'acc': 'accuracy',
           'precision': 'precision_weighted',
           'recall': 'recall_macro',
           'f1_score' : 'f1_macro'}
SVM_scores = cross_validate(svm_clf, new_vect, ds['Label'], scoring=scoring,
                         cv=10, return_train_score=True)

In [96]:
print(SVM_scores.keys())
print(SVM_scores['test_acc'].mean())
print(SVM_scores['test_precision'].mean())
print(SVM_scores['test_recall'].mean())
print(SVM_scores['test_f1_score'].mean())

dict_keys(['fit_time', 'score_time', 'test_acc', 'train_acc', 'test_precision', 'train_precision', 'test_recall', 'train_recall', 'test_f1_score', 'train_f1_score'])
0.7012225153917494
0.7206748198846014
0.7282451499118167
0.7254074115710943


In [120]:
#Use cross validation predict to make prediction on the entire dataset
SVM_pred = cross_val_predict(svm_clf, new_vect, y1, cv=10)

#Use classification report to get the precision & recall table, compare with prediction and actual labels
print(classification_report(y1,SVM_pred))

             precision    recall  f1-score   support

          0       0.59      0.66      0.62        74
          1       0.67      0.71      0.69        84
          2       0.68      0.64      0.66        77
          3       0.86      0.82      0.84        61
          4       0.79      0.89      0.84        37
          5       0.73      0.79      0.76        42
          6       0.80      0.87      0.83        45
          7       0.87      0.79      0.83        34
          8       0.61      0.49      0.54        74

avg / total       0.71      0.71      0.71       528



# SVM Non-Linear (New Data)

In [97]:
svm_non=SVC(kernel="rbf",cache_size=1,class_weight={1:2}).fit(new_vect,ds['Label'])

In [98]:
scoring = {'acc': 'accuracy',
           'precision': 'precision_weighted',
           'recall': 'recall_macro',
           'f1_score' : 'f1_macro'}
NON_scores = cross_validate(svm_non, new_vect, ds['Label'], scoring=scoring,
                         cv=10, return_train_score=True)

/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classif

/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [99]:
print(NON_scores.keys())
print(NON_scores['test_acc'].mean())
print(NON_scores['test_precision'].mean())
print(NON_scores['test_recall'].mean())
print(NON_scores['test_f1_score'].mean())

dict_keys(['fit_time', 'score_time', 'test_acc', 'train_acc', 'test_precision', 'train_precision', 'test_recall', 'train_recall', 'test_f1_score', 'train_f1_score'])
0.1591863284473359
0.025351455039253217
0.11111111111111112
0.030515277268575898


In [121]:
#Use cross validation predict to make prediction on the entire dataset
NON_pred = cross_val_predict(svm_non, new_vect, y1, cv=10)

#Use classification report to get the precision & recall table, compare with prediction and actual labels
print(classification_report(y1,NON_pred))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        74
          1       0.16      1.00      0.27        84
          2       0.00      0.00      0.00        77
          3       0.00      0.00      0.00        61
          4       0.00      0.00      0.00        37
          5       0.00      0.00      0.00        42
          6       0.00      0.00      0.00        45
          7       0.00      0.00      0.00        34
          8       0.00      0.00      0.00        74

avg / total       0.03      0.16      0.04       528



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
